In [1]:
%load_ext autoreload
%autoreload 2

import math

# The deformation module library is not automatically installed yet, we need to add its path manually
import sys
sys.path.append("../../")

import matplotlib.pyplot as plt
import torch

import implicitmodules.torch as dm

In [25]:
N = 10
extend = 1.
sigma_noise = 0.00
source = torch.zeros(2*N, 2)
source[:N, 0] = torch.linspace(-extend/2., extend/2., N)
source[N:, 1] = torch.linspace(-extend/2., extend/2., N)
source = source + sigma_noise*torch.randn_like(source)

lineardef = torch.mm(dm.Utilities.rot2d(2*math.pi/3), 1.*torch.tensor([[1., 0.], [0., 1.]]))
target = torch.bmm(source.view(-1, 1, 2), lineardef.transpose(0,1).repeat(2*N, 1, 1)).view(-1, 2)
target = target + sigma_noise*torch.randn_like(target)

In [26]:
plt.plot(source[:, 0].numpy(), source[:, 1].numpy(), '-', color='blue')
plt.plot(target[:, 0].numpy(), target[:, 1].numpy(), '-', color='red')
plt.axis('equal')
plt.show()

In [27]:
source[0]

tensor([-0.5000,  0.0000])

In [79]:
#lineardefmodule = dm.DeformationModules.LinearDeformation.build(torch.eye(2, requires_grad=True), gd=torch.zeros(1, 2, requires_grad=True))
#model = dm.Models.ModelPointsRegistration([source.clone()], [lineardefmodule], [dm.Attachment.EuclideanPointwiseDistanceAttachment()], other_parameters=[lineardefmodule.A], lam=100., fit_moments=True)

#lineardefmodule = dm.DeformationModules.LocalRotation.build(2, 1, 1., coeff=1., gd=torch.zeros(1, 2, requires_grad=True))

sigtensor = torch.tensor(1., requires_grad=True)
lineardefmodule = dm.DeformationModules.LocalLinear.build(2, torch.eye(2, requires_grad=True), sigtensor, coeff=1., gd=torch.zeros(1, 2, requires_grad=True))


model = dm.Models.ModelPointsRegistration([source.clone()], [lineardefmodule], [dm.Attachment.EuclideanPointwiseDistanceAttachment()], other_parameters=[lineardefmodule.A, sigtensor], lam=100., fit_moments=True)

In [80]:
lineardefmodule.sigma

tensor(1., requires_grad=True)

In [81]:
shoot_meth = 'rk4'
modelfitter = dm.Models.ModelFittingScipy(model, 1.)
costs = modelfitter.fit([target.clone()], 55, options={'shoot_method': shoot_meth})

Initial energy = 962.250
Iteration: 1 
Total energy = 753.892273 
Attach cost = 753.476746 
Deformation cost = 0.415551
Iteration: 2 
Total energy = 555.318054 
Attach cost = 554.695984 
Deformation cost = 0.622068
Iteration: 3 
Total energy = 555.090881 
Attach cost = 554.447083 
Deformation cost = 0.643773
Iteration: 4 
Total energy = 551.502563 
Attach cost = 551.277039 
Deformation cost = 0.225539
Iteration: 5 
Total energy = 551.415161 
Attach cost = 551.150940 
Deformation cost = 0.264205
Iteration: 6 
Total energy = 551.408813 
Attach cost = 551.262146 
Deformation cost = 0.146678
Iteration: 7 
Total energy = 551.363892 
Attach cost = 551.189880 
Deformation cost = 0.174015
Iteration: 8 
Total energy = 551.338562 
Attach cost = 551.142822 
Deformation cost = 0.195711
Iteration: 9 
Total energy = 551.302124 
Attach cost = 551.116150 
Deformation cost = 0.185969
Iteration: 10 
Total energy = 551.189270 
Attach cost = 551.051208 
Deformation cost = 0.138065
Iteration: 11 
Total ene

Iteration: 48 
Total energy = 49.526997 
Attach cost = 20.536568 
Deformation cost = 28.990429
Iteration: 49 
Total energy = 43.238853 
Attach cost = 15.747263 
Deformation cost = 27.491589
Iteration: 50 
Total energy = 38.554695 
Attach cost = 12.342288 
Deformation cost = 26.212406
Iteration: 51 
Total energy = 36.080513 
Attach cost = 10.041378 
Deformation cost = 26.039135
Iteration: 52 
Total energy = 34.107613 
Attach cost = 7.491438 
Deformation cost = 26.616173
Iteration: 53 
Total energy = 34.042526 
Attach cost = 7.398759 
Deformation cost = 26.643766
Iteration: 54 
Total energy = 34.003746 
Attach cost = 7.394913 
Deformation cost = 26.608833
Iteration: 55 
Total energy = 33.941410 
Attach cost = 7.322059 
Deformation cost = 26.619352
Optimisation process exited with message: b'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT'
Final energy = 33.941410064697266
Closure evaluations = 97
Time elapsed = 52.308082580566406


In [82]:
compound = dm.DeformationModules.CompoundModule(model.modules)
man = model.init_manifold.clone()
#co1 = torch.zeros_like(source)
#co1[0,1] = -10
#man[0].fill_cotan(co1)
compound.manifold.fill(man)


In [83]:
#shoot_meth = 'rk4'
intermediate_states_opt, intermediate_controls_opt = dm.HamiltonianDynamic.shoot(dm.HamiltonianDynamic.Hamiltonian(compound), 10, shoot_meth, intermediates=True)

In [84]:
len(intermediate_controls_opt)

40

In [85]:
t = -1


cotan0 = model.init_manifold.cotan[0]
cotan1 = model.init_manifold.cotan[1]
lindef_opti = model.modules[1].A

print(control)
print(cotan0)
print(cotan1)
print(lindef_opti)

%matplotlib qt5
plt.plot(source[:, 0].numpy(), source[:, 1].numpy(), 'o', color='blue')
plt.plot(target[:, 0].numpy(), target[:, 1].numpy(), 'o', color='red')
#plt.plot(deformed_source[:, 0].numpy(), deformed_source[:, 1].numpy(), 'x', color='green')
for t in range(11):
    deformed_source = intermediate_states_opt[t].gd[0].detach()
    plt.plot(deformed_source[:, 0].numpy(), deformed_source[:, 1].numpy(), '.', color='green')
plt.plot(deformed_source[:, 0].numpy(), deformed_source[:, 1].numpy(), 'x', color='green')    
    
#for i in range(source.shape[0]):
#    plt.plot([source[i, 0].numpy(), deformed_source[i, 0].numpy()],
#             [source[i, 1].numpy(), deformed_source[i, 1].numpy()], color='black')

plt.axis('equal')
plt.show()


[tensor([]), tensor(-0.1223)]
tensor([[-14.7312,  -4.1077],
        [-12.2527,  -5.4001],
        [ -9.4240,  -7.3860],
        [ -5.7431,  -5.0788],
        [ -1.8445,  -1.7505],
        [  1.7370,   1.6591],
        [  4.8438,   4.5254],
        [  8.2038,   5.9924],
        [ 12.0033,   4.6170],
        [ 15.5251,   1.7673],
        [  2.9518, -13.8637],
        [  5.7907, -11.0358],
        [  8.1420,  -8.1355],
        [  5.4337,  -5.3011],
        [  1.7513,  -1.9322],
        [ -1.6537,   2.0101],
        [ -4.3445,   5.9116],
        [ -5.6973,   9.3847],
        [ -4.3297,  12.1094],
        [ -2.7734,  14.5751]], requires_grad=True)
tensor([[-2.7807e-06,  7.8163e-07]], requires_grad=True)
tensor([[-1.6111e-03,  4.7351e+00],
        [-4.7337e+00,  6.1912e-04]], requires_grad=True)


In [86]:
lineardefmodule.sigma

tensor(2.3670, requires_grad=True)

In [87]:
intermediate_controls_opt

[[tensor([]), tensor(-2.8433)],
 [tensor([]), tensor(-2.8567)],
 [tensor([]), tensor(-2.8173)],
 [tensor([]), tensor(-2.8782)],
 [tensor([]), tensor(-2.8433)],
 [tensor([]), tensor(-2.8567)],
 [tensor([]), tensor(-2.8173)],
 [tensor([]), tensor(-2.8783)],
 [tensor([]), tensor(-2.8433)],
 [tensor([]), tensor(-2.8567)],
 [tensor([]), tensor(-2.8174)],
 [tensor([]), tensor(-2.8782)],
 [tensor([]), tensor(-2.8433)],
 [tensor([]), tensor(-2.8567)],
 [tensor([]), tensor(-2.8174)],
 [tensor([]), tensor(-2.8782)],
 [tensor([]), tensor(-2.8433)],
 [tensor([]), tensor(-2.8567)],
 [tensor([]), tensor(-2.8174)],
 [tensor([]), tensor(-2.8782)],
 [tensor([]), tensor(-2.8433)],
 [tensor([]), tensor(-2.8567)],
 [tensor([]), tensor(-2.8175)],
 [tensor([]), tensor(-2.8782)],
 [tensor([]), tensor(-2.8433)],
 [tensor([]), tensor(-2.8567)],
 [tensor([]), tensor(-2.8175)],
 [tensor([]), tensor(-2.8782)],
 [tensor([]), tensor(-2.8433)],
 [tensor([]), tensor(-2.8568)],
 [tensor([]), tensor(-2.8175)],
 [tensor

In [81]:
lineardefmodule

[tensor([], requires_grad=True), tensor(112.0549, grad_fn=<DivBackward0>)]

In [91]:
intermediate_controls_opt

[[tensor([]), tensor(112.0549)],
 [tensor([]), tensor(1675.3818)],
 [tensor([]), tensor(46747552.)],
 [tensor([]), tensor(1.0216e+21)],
 [tensor([]), tensor(inf)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan)],
 [tensor([]), tensor(nan